In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib
import matplotlib.pyplot as plt
from umap import UMAP
import sklearn
import seaborn as sns
from COSMOS import cosmos
from COSMOS.pyWNN import pyWNN 
import warnings
warnings.filterwarnings('ignore')
random_seed = 20

In [2]:
path="../datasets/10x_human_lymph_node_D1/"
adata2=sc.read_h5ad(path+"adata_ADT.h5ad")
adata1=sc.read_h5ad(path+"adata_RNA.h5ad")

adata1.obs['x_pos'] = adata1.obsm['spatial'][:,0]
adata1.obs['y_pos'] = adata1.obsm['spatial'][:,1]

adata2.obs['x_pos'] = adata2.obsm['spatial'][:,0]
adata2.obs['y_pos'] = adata2.obsm['spatial'][:,1]

# COSMOS integration
cosmos_comb = cosmos.Cosmos(adata1=adata1,adata2=adata2)
cosmos_comb.preprocessing_data(n_neighbors = 10)
cosmos_comb.train(spatial_regularization_strength=0, z_dim=50, 
         lr=1e-3, wnn_epoch = 500, total_epoch=1000, max_patience_bef=10, max_patience_aft=30, min_stop=200, 
         random_seed=random_seed, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)

adata1.obsm["COSMOS"]=cosmos_comb.embedding
adata1.write_h5ad(path.replace("datasets","results")+"adata_COSMOS.h5ad")